<a href="https://colab.research.google.com/github/PamelaVQ/Base-ML/blob/master/Pytorch_Basics/pytorch_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create a simple autoencoder using pytorch

In [15]:
import torch
from torchvision.datasets import MNIST
from torch import nn
from torchvision import transforms
from torch.utils.data import dataloader
from torch.autograd import Variable

In [2]:
dataset = MNIST(root='.', download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]))
dataset

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Dataset MNIST
    Number of datapoints: 60000
    Root location: .
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )

In [28]:
# create a dataloader
batch_size = 128
mnist_data = dataloader.DataLoader(dataset, batch_size=batch_size, shuffle=True)
num_epochs = 20
print(dataset.data.shape[1], dataset.data.shape[2])

28 28


In [29]:
# create a simple autoencoder
class Encoder(nn.Module):
  def __init__(self, input_1, input_2, batch_size):
    super(Encoder, self).__init__()
    self.linear1 = nn.Linear(input_1 * input_2, batch_size)
    self.linear2 = nn.Linear(batch_size, 64)
    self.linear3 = nn.Linear(64, 12)
    self.linear4 = nn.Linear(12, 3)

  def forward(self, x):
    x = torch.relu(self.linear1(x))
    x = torch.relu(self.linear2(x))
    x = torch.relu(self.linear3(x))
    x = torch.relu(self.linear4(x))
    x = nn.Tanh(x)
    return x

class Decoder(nn.Module):
  def __init__(self, output_1, output_2, batch_size):
    super(Decoder, self).__init__()
    self.linear1 = nn.Linear(3, 12)
    self.linear2 = nn.Linear(12, 64)
    self.linear3 = nn.Linear(64, batch_size)
    self.linear4 = nn.Linear(batch_size, output_1 * output_2)   

  def forward(self, x):
    x = torch.relu(self.linear1(x))
    x = torch.relu(self.linear2(x))
    x = torch.relu(self.linear3(x))
    x = torch.relu(self.linear4(x))
    x = nn.Tanh(x)
    return x


In [30]:
encoder = Encoder(dataset.data.shape[1], dataset.data.shape[2], batch_size)
decoder = Decoder(dataset.data.shape[1], dataset.data.shape[2], batch_size)

autoencoder = nn.Sequential(encoder, decoder)

In [37]:
x = Variable(torch.ones(1, 1, 784, 128))
e = encoder(x)
d = decoder(e)

print('Input\t ', list(x.data.shape))
print('Embedding', list(e.data.shape))
print('Output\t ', list(d.data.shape))

RuntimeError: ignored

In [31]:
loss_fn = nn.MSELoss()
optim = torch.optim.Adam(autoencoder.parameters())
autoencoder.train()

Sequential(
  (0): Encoder(
    (linear1): Linear(in_features=784, out_features=128, bias=True)
    (linear2): Linear(in_features=128, out_features=64, bias=True)
    (linear3): Linear(in_features=64, out_features=12, bias=True)
    (linear4): Linear(in_features=12, out_features=3, bias=True)
  )
  (1): Decoder(
    (linear1): Linear(in_features=3, out_features=12, bias=True)
    (linear2): Linear(in_features=12, out_features=64, bias=True)
    (linear3): Linear(in_features=64, out_features=128, bias=True)
    (linear4): Linear(in_features=128, out_features=784, bias=True)
  )
)

In [32]:
for epoch in range(num_epochs):
  for batch_num, (data, _) in enumerate(mnist_data):
    optim.zero_grad()
    data = data.view(data.shape[0], -1)
    print(data.shape)
    data = Variable(data)
    print(data)
    output = autoencoder(data)
    print(output)
    break

torch.Size([128, 784])
tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])


TypeError: ignored